In [ ]:
import pandas as pd
from sklearn import preprocessing

In [ ]:
holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv', index_col='date')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv', index_col='date')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv', index_col='id')
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv', index_col='id')
transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
test.head()

In [ ]:
y = train.sales
y = pd.DataFrame(y)
train1 = train.drop(['sales'], axis=1)

In [ ]:
train1.head()

In [ ]:
y.head()

In [ ]:
stores.head()

In [ ]:
train2 = pd.merge(train1, stores, how="left", on="store_nbr")
test2 = pd.merge(test, stores, how="left", on="store_nbr")

In [ ]:
train2.head()

In [ ]:
#transactions.head()

In [ ]:
#train3 = pd.merge(train2, transactions, how="left", on="store_nbr")

In [ ]:
#train3.head()

extract date features

In [ ]:
def extract_weekday(s):
    return s.dayofweek

def extract_monthday(s):
    return s.day

def extract_month(s):
    return s.month

def extract_year(s):
    return s.year

In [ ]:
train2['date'] = pd.to_datetime(train2['date'])
train2['weekday'] = train2['date'].apply(extract_weekday)
train2['extract_monthday'] = train2['date'].apply(extract_monthday)
train2['year'] = train2['date'].apply(extract_year)
train2['month'] = train2['date'].apply(extract_month)

test2['date'] = pd.to_datetime(test2['date'])
test2['weekday'] = test2['date'].apply(extract_weekday)
test2['extract_monthday'] = test2['date'].apply(extract_monthday)
test2['year'] = test2['date'].apply(extract_year)
test2['month'] = test2['date'].apply(extract_month)

In [ ]:
train2.head()

In [ ]:
train2.dtypes

In [ ]:
cat_cols = [cname for cname in train2.columns if
                    train2[cname].dtype == 'object']

In [ ]:
train2.nunique()

In [ ]:
cat_cols

In [ ]:
enc = preprocessing.LabelEncoder()
for col in cat_cols:
    train2[col] = enc.fit_transform(train2[col])
    test2[col] = enc.fit_transform(test2[col])

In [ ]:
train2

In [ ]:
'''
for col in train2.columns:
    if train2[col].dtype == 'float64':
        train2[col] = train2[col].astype('float16')
        test2[col] = test2[col].astype('float32')
    if dataset_train[col].dtype == 'int64':
        dataset_train[col] = dataset_train[col].astype('int8')
        dataset_test1[col] = dataset_test1[col].astype('int8')
'''

In [ ]:
import tensorflow as tf

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
train3 = train2.drop('date', axis=1)
test3 = test2.drop('date', axis=1)

In [ ]:
train3.head()

In [ ]:
import numpy as np
train_data = np.array(train3)
test_data = np.array(test3)

In [ ]:
train_data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
dataset_train_sc = x_scaler.fit_transform(train_data)
dataset_test_sc = x_scaler.transform(test_data)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from tensorflow.keras.engine.input_layer import Input
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
def model_builder(lr):
    """Модель нейронной сети"""
    inputA = keras.Input(shape=(11))
    line = Reshape((11,1))(inputA)
    line = Conv1D(filters=16, kernel_size=2, padding='same', activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.1)(line)

    line = Conv1D(filters=16, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.3)(line)
    
    line = Conv1D(filters=32, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    #line = MaxPooling1D(pool_size=2)(line)
    
    line = Conv1D(filters=32, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    line = Conv1D(filters=64, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    line = Conv1D(filters=64, kernel_size=3, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.5)(line)
    
    #line = MaxPooling1D(pool_size=2)(line)
    
    line = Flatten()(line)
    line = Dense(64, activation='relu')(line)
    line = Dropout(0.5)(line)
    #line = Dense(64, activation='relu')(line)
    outputA = Dense(units=1)(line)
    model = Model(inputs=inputA, outputs=outputA)
    #model = keras.models.load_model('models/model2')
    model.compile(loss = 'mse', optimizer = Adam(lr=lr), metrics=['mae'],)
    return model

In [ ]:
lr=0.001
with strategy.scope():
    model = model_builder(lr)

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = 'best.h5'
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

In [ ]:
#model = load_model('../input/tps1021/best_85221.h5')

preds = model.predict(dataset_test_sc)

In [ ]:
N_split = int(0.2 * len(dataset_train_sc))
dataset_sc_TRAIN = dataset_train_sc[:-N_split, :]
dataset_sc_VAL = dataset_train_sc[-N_split:, :]
y_TRAIN = y[:-N_split]
y_VAL = y[-N_split:]

import numpy as np
big_dataset_X = np.concatenate([dataset_sc_TRAIN, dataset_test_sc], axis=0)
len(big_dataset_X)

big_dataset_y = np.concatenate([y_TRAIN, preds], axis=0)
len(big_dataset_y)

N = len(preds)/(len(preds)+len(y_TRAIN))
N

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001, verbose=1, mode='min')

pseudo training

EPOCHS = 50
#EPOCHS = 1
model.fit(
    big_dataset_X, big_dataset_y,
    validation_data=(dataset_sc_VAL, y_VAL), epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], batch_size=512, shuffle=True)

real training

In [ ]:
EPOCHS = 200
#EPOCHS = 1
model.fit(
    dataset_sc_TRAIN, y_TRAIN,
    validation_data=(dataset_sc_VAL, y_VAL), epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], batch_size=4096, shuffle=True)

repeat few times 2 cells above

In [ ]:
model = load_model('best.h5')

In [ ]:
preds = model.predict(dataset_test_sc)
output = pd.DataFrame({'Id': test.index,'sales': preds[:,0]})
path = 'sample_submission.csv'
output.to_csv(path, index=False)
output 